# Chương 1.1 Bài Tập: Cơ Bản VPA với Dữ liệu Thực tế

**Mục Tiêu:**
- Thực hành nhận diện tín hiệu VPA với dữ liệu thị trường Việt Nam thực tế
- Học cách tính toán các chỉ báo VPA từ dữ liệu OHLCV thô
- So sánh phân tích tự động với insights VPA chuyên gia
- Xây dựng nền tảng cho các kỹ thuật VPA nâng cao

**Bộ Dữ Liệu Sử Dụng:**
- `market_data/VCB_2025-01-02_to_2025-07-21.csv`
- `market_data/VNINDEX_2025-01-02_to_2025-07-21.csv`  
- `vpa_data/VCB.md` và `vpa_data/VNINDEX.md`

## Thiết Lập và Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 8)

print("Libraries imported successfully! 🚀")
print("Ready to analyze Vietnam stock market với VPA methodology")

## Bài Tập 1: Tải Dữ Liệu và Phân Tích Cơ Bản

In [ ]:
# Load VCB data
def load_stock_data(ticker, data_folder='../../../market_data'):
    """Load stock data from CSV files"""
    file_path = f'{data_folder}/{ticker}_2025-01-02_to_2025-07-21.csv'
    df = pd.read_csv(file_path)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df

# Load VCB and VNINDEX data
vcb_data = load_stock_data('VCB')
vnindex_data = load_stock_data('VNINDEX')

print("📊 VCB Data Summary:")
print(f"Date range: {vcb_data.index[0]} to {vcb_data.index[-1]}")
print(f"Total trading days: {len(vcb_data)}")
print(f"Average volume: {vcb_data['volume'].mean():,.0f}")
print(f"Price range: {vcb_data['low'].min():.2f} - {vcb_data['high'].max():.2f}")

print("\n📈 VNINDEX Data Summary:")
print(f"Date range: {vnindex_data.index[0]} to {vnindex_data.index[-1]}")
print(f"Total trading days: {len(vnindex_data)}")
print(f"Average volume: {vnindex_data['volume'].mean():,.0f}")
print(f"Index range: {vnindex_data['low'].min():.2f} - {vnindex_data['high'].max():.2f}")

### 💡 Nhiệm Vụ 1.1: Khám Phá Dữ Liệu Cơ Bản

**Các nhiệm vụ của bạn:**
1. Hiển thị 5 dòng đầu tiên của dữ liệu VCB
2. Tính toán thống kê cơ bản (trung bình, độ lệch chuẩn, min, max) cho khối lượng và giá đóng cửa
3. Tìm ngày có khối lượng cao nhất và thấp nhất

In [ ]:
# YOUR CODE HERE
# Task 1: Display first 5 rows
print("First 5 rows of VCB data:")
# ...

# Task 2: Basic statistics
print("\nBasic statistics:")
# ...

# Task 3: Find extreme volume days
print("\nExtreme volume days:")
# ...

## Bài Tập 2: Tính Toán Chỉ Báo VPA

In [ ]:
def calculate_vpa_indicators(df):
    """Calculate essential VPA indicators"""
    df = df.copy()
    
    # Price indicators
    df['price_change'] = df['close'].pct_change() * 100
    df['price_range'] = df['high'] - df['low']  
    df['price_spread'] = (df['high'] - df['low']) / df['close'] * 100
    
    # Volume indicators
    df['volume_ma5'] = df['volume'].rolling(window=5).mean()
    df['volume_ma20'] = df['volume'].rolling(window=20).mean()
    df['volume_ratio'] = df['volume'] / df['volume_ma20']
    
    # Close position (where price closed within the day's range)
    df['close_position'] = (df['close'] - df['low']) / (df['high'] - df['low'])
    df['close_position'] = df['close_position'].fillna(0.5)  # Handle zero range days
    
    # VPA signal flags
    df['high_volume'] = df['volume_ratio'] > 1.5
    df['ultra_high_volume'] = df['volume_ratio'] > 2.0
    df['low_volume'] = df['volume_ratio'] < 0.7
    
    # Close position categories
    df['bullish_close'] = df['close_position'] > 0.7  # Close near high
    df['bearish_close'] = df['close_position'] < 0.3  # Close near low
    
    return df

# Apply VPA indicators to our data
vcb_vpa = calculate_vpa_indicators(vcb_data)
vnindex_vpa = calculate_vpa_indicators(vnindex_data)

print("✅ VPA indicators calculated!")
print("\nSample VPA indicators for VCB:")
display_cols = ['close', 'volume', 'volume_ratio', 'close_position', 'price_change']
print(vcb_vpa[display_cols].tail())

### 💡 Nhiệm Vụ 2.1: Phát Hiện Tín Hiệu VPA

**Nhận diện các mẫu hình VPA cụ thể từ Chương 1.1:**

In [ ]:
# Find No Supply signals
# Criteria: Low volume (<0.8x average) + price up or flat + close near high
no_supply_signals = vcb_vpa[
    (vcb_vpa['volume_ratio'] < 0.8) & 
    (vcb_vpa['price_change'] >= -0.5) &  # Price flat or slightly up
    (vcb_vpa['close_position'] > 0.6)    # Close in upper part of range
]

print(f"🔍 No Supply signals found: {len(no_supply_signals)}")
if len(no_supply_signals) > 0:
    print("\nTop No Supply signals:")
    cols = ['close', 'volume', 'volume_ratio', 'price_change', 'close_position']
    print(no_supply_signals[cols].head())

# Find Stopping Volume signals  
# Criteria: Ultra high volume (>2x) + bullish close + price recovery
stopping_volume = vcb_vpa[
    (vcb_vpa['volume_ratio'] > 2.0) &
    (vcb_vpa['close_position'] > 0.6) &
    (vcb_vpa['price_change'] > 0)  # Price up on the day
]

print(f"\n⚡ Stopping Volume signals found: {len(stopping_volume)}")
if len(stopping_volume) > 0:
    print("\nTop Stopping Volume signals:")
    print(stopping_volume[cols].head())

### 💡 Nhiệm Vụ 2.2: Phân Tích Effort vs Result

**Tìm bất thường Effort vs Result của VNINDEX như case study ngày 15/05/2025:**

In [ ]:
# YOUR CODE HERE
# Find days where high volume doesn't produce proportional price movement
effort_result_anomalies = vnindex_vpa[
    # HIGH EFFORT: Volume ratio > 1.8
    # LOW RESULT: Absolute price change < 0.5%
    # Add your conditions here...
]

print(f"⚠️ Effort vs Result Anomalies found: {len(effort_result_anomalies)}")
if len(effort_result_anomalies) > 0:
    print("\nEffort vs Result Anomalies:")
    cols = ['close', 'volume', 'volume_ratio', 'price_change', 'price_spread']
    print(effort_result_anomalies[cols])
    
    # Check if May 15, 2025 anomaly is in our detected signals
    may_15 = '2025-05-15'
    if may_15 in effort_result_anomalies.index:
        print(f"\n✅ Successfully detected the May 15, 2025 anomaly!")
        print(f"Details: {effort_result_anomalies.loc[may_15][cols]}")
    else:
        print(f"\n❌ May 15, 2025 anomaly not detected. Check your criteria.")

## Bài Tập 3: Phân Tích Trực Quan

In [ ]:
def plot_vpa_analysis(df, ticker, start_date=None, end_date=None, highlight_signals=True):
    """Create comprehensive VPA chart"""
    
    # Filter date range if specified
    if start_date and end_date:
        df_plot = df.loc[start_date:end_date].copy()
    else:
        df_plot = df.copy()
    
    fig, axes = plt.subplots(3, 1, figsize=(16, 12), height_ratios=[3, 1, 1])
    
    # 1. Price Chart với VPA signals
    ax1 = axes[0]
    ax1.plot(df_plot.index, df_plot['close'], linewidth=2, color='blue', label='Close Price')
    
    if highlight_signals:
        # Highlight volume spikes
        high_vol = df_plot[df_plot['volume_ratio'] > 1.8]
        ax1.scatter(high_vol.index, high_vol['close'], color='red', s=60, 
                   label='High Volume', alpha=0.7, marker='^')
        
        # Highlight low volume tests
        low_vol = df_plot[df_plot['volume_ratio'] < 0.7]
        ax1.scatter(low_vol.index, low_vol['close'], color='green', s=40,
                   label='Low Volume', alpha=0.7, marker='v')
    
    ax1.set_title(f'{ticker} - VPA Analysis', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Price', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Volume Chart
    ax2 = axes[1]
    bars = ax2.bar(df_plot.index, df_plot['volume'], alpha=0.6, color='lightblue')
    
    # Color-code volume bars
    for i, bar in enumerate(bars):
        if df_plot['volume_ratio'].iloc[i] > 2.0:
            bar.set_color('red')  # Ultra high volume
        elif df_plot['volume_ratio'].iloc[i] > 1.5:
            bar.set_color('orange')  # High volume
        elif df_plot['volume_ratio'].iloc[i] < 0.7:
            bar.set_color('green')  # Low volume
    
    ax2.axhline(y=df_plot['volume_ma20'].mean(), color='blue', linestyle='--', 
               label='20-day MA', alpha=0.8)
    ax2.set_ylabel('Volume', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Volume Ratio Chart
    ax3 = axes[2]
    ax3.plot(df_plot.index, df_plot['volume_ratio'], linewidth=2, color='purple', 
            label='Volume Ratio')
    ax3.axhline(y=1.0, color='blue', linestyle='-', alpha=0.5, label='Average')
    ax3.axhline(y=1.5, color='orange', linestyle='--', alpha=0.7, label='High Threshold')
    ax3.axhline(y=2.0, color='red', linestyle='--', alpha=0.7, label='Ultra High Threshold')
    ax3.axhline(y=0.7, color='green', linestyle='--', alpha=0.7, label='Low Threshold')
    
    ax3.set_ylabel('Volume Ratio', fontsize=12)
    ax3.set_xlabel('Date', fontsize=12)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Format x-axis dates
    for ax in axes:
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

# Plot VCB analysis for June 2025 (contains the Stopping Volume example)
plot_vpa_analysis(vcb_vpa, 'VCB', '2025-06-01', '2025-07-01')

### 💡 Nhiệm Vụ 3.1: Nhận Dạng Mẫu Hình Trực Quan

**Nhìn vào biểu đồ trên, hãy xác định:**
1. Ngày nào có volume spike đáng kể nhất?
2. Có pattern nào giống "Stopping Volume" không?
3. Phản ứng giá sau những ngày khối lượng cao như thế nào?

In [ ]:
# YOUR ANALYSIS HERE
# Find the day with highest volume ratio in June 2025
june_data = vcb_vpa['2025-06-01':'2025-07-01']
max_volume_day = june_data.loc[june_data['volume_ratio'].idxmax()]

print("📊 Highest volume day in June 2025:")
print(f"Date: {june_data['volume_ratio'].idxmax()}")
print(f"Volume ratio: {max_volume_day['volume_ratio']:.2f}x")
print(f"Price change: {max_volume_day['price_change']:.2f}%")
print(f"Close position: {max_volume_day['close_position']:.2f}")

# Check if this matches the June 13 example từ vpa_data/VCB.md
june_13 = '2025-06-13'
if june_13 in june_data.index:
    june_13_data = june_data.loc[june_13]
    print(f"\n📅 June 13, 2025 analysis:")
    print(f"Volume ratio: {june_13_data['volume_ratio']:.2f}x")
    print(f"Price change: {june_13_data['price_change']:.2f}%")
    print(f"Close position: {june_13_data['close_position']:.2f}")
    
    if (june_13_data['volume_ratio'] > 2.0 and 
        june_13_data['close_position'] > 0.6 and 
        june_13_data['price_change'] > 0):
        print("✅ This matches Stopping Volume criteria!")
    else:
        print("❌ This doesn't match typical Stopping Volume pattern")

## Bài Tập 4: Tham Chiếu Chéo với Phân Tích Chuyên Gia

In [ ]:
# Create comparison table between our automated analysis và expert insights
def create_signal_comparison(df, expert_dates_list):
    """Compare automated signals với expert-identified dates"""
    
    comparison_data = []
    
    for date_str in expert_dates_list:
        if date_str in df.index:
            row = df.loc[date_str]
            
            # Automated signal detection
            auto_signals = []
            
            if row['volume_ratio'] > 2.0 and row['close_position'] > 0.6:
                auto_signals.append('Stopping Volume')
            elif row['volume_ratio'] < 0.8 and abs(row['price_change']) < 0.5:
                auto_signals.append('No Supply')
            elif row['volume_ratio'] > 1.8 and abs(row['price_change']) < 0.3:
                auto_signals.append('Effort vs Result')
            else:
                auto_signals.append('No Clear Signal')
            
            comparison_data.append({
                'Date': date_str,
                'Close': row['close'],
                'Volume_Ratio': f"{row['volume_ratio']:.2f}x",
                'Price_Change': f"{row['price_change']:.2f}%",
                'Close_Position': f"{row['close_position']:.2f}",
                'Auto_Signal': ', '.join(auto_signals)
            })
    
    return pd.DataFrame(comparison_data)

# Expert-identified dates từ vpa_data/VCB.md
expert_dates = [
    '2025-06-13',  # Stopping Volume example
    '2025-06-16',  # Test for Supply
    '2025-06-19',  # No Supply
    '2025-06-20',  # Effort to Rise
]

comparison_table = create_signal_comparison(vcb_vpa, expert_dates)
print("📋 Automated vs Expert Analysis Comparison:")
print(comparison_table.to_string(index=False))

# Calculate accuracy metrics
total_expert_dates = len(expert_dates)
automated_matches = len(comparison_table[comparison_table['Auto_Signal'] != 'No Clear Signal'])

print(f"\n📊 Analysis Results:")
print(f"Expert-identified dates: {total_expert_dates}")
print(f"Automated signal detection: {automated_matches}/{total_expert_dates}")
print(f"Detection rate: {(automated_matches/total_expert_dates)*100:.1f}%")

## Bài Tập 5: So Sánh Đa Cổ Phiếu

In [ ]:
# YOUR CODE HERE
# Load and analyze multiple stocks for VPA patterns

# Banking sector comparison
banking_tickers = ['VCB', 'TCB', 'STB', 'MBB']
banking_analysis = {}

for ticker in banking_tickers:
    try:
        # Load data
        data = load_stock_data(ticker)
        vpa_data = calculate_vpa_indicators(data)
        
        # Count signals
        stopping_volume_count = len(vpa_data[
            (vpa_data['volume_ratio'] > 2.0) & 
            (vpa_data['close_position'] > 0.6) & 
            (vpa_data['price_change'] > 0)
        ])
        
        no_supply_count = len(vpa_data[
            (vpa_data['volume_ratio'] < 0.8) & 
            (vpa_data['price_change'] >= -0.5) &
            (vpa_data['close_position'] > 0.6)
        ])
        
        banking_analysis[ticker] = {
            'stopping_volume': stopping_volume_count,
            'no_supply': no_supply_count,
            'avg_volume': data['volume'].mean(),
            'price_volatility': vpa_data['price_change'].std()
        }
        
    except Exception as e:
        print(f"Error loading {ticker}: {e}")
        continue

# Create comparison DataFrame
banking_df = pd.DataFrame(banking_analysis).T
print("🏦 Banking Sector VPA Comparison:")
print(banking_df)

# Find most active stock (most VPA signals)
banking_df['total_signals'] = banking_df['stopping_volume'] + banking_df['no_supply']
most_active = banking_df['total_signals'].idxmax()
print(f"\n🏆 Most VPA-active banking stock: {most_active}")
print(f"Total signals: {banking_df.loc[most_active, 'total_signals']}")

## Bài Tập 6: Theo Dõi Hiệu Suất

In [ ]:
def backtest_vpa_signals(df, signal_type='stopping_volume', holding_days=5):
    """Simple backtest của VPA signals"""
    
    if signal_type == 'stopping_volume':
        signals = df[
            (df['volume_ratio'] > 2.0) & 
            (df['close_position'] > 0.6) & 
            (df['price_change'] > 0)
        ].copy()
    elif signal_type == 'no_supply':
        signals = df[
            (df['volume_ratio'] < 0.8) & 
            (df['price_change'] >= -0.5) &
            (df['close_position'] > 0.6)
        ].copy()
    
    results = []
    
    for signal_date in signals.index:
        entry_price = df.loc[signal_date, 'close']
        
        # Find exit date (holding_days later)
        try:
            signal_idx = df.index.get_loc(signal_date)
            if signal_idx + holding_days < len(df):
                exit_date = df.index[signal_idx + holding_days]
                exit_price = df.loc[exit_date, 'close']
                
                return_pct = (exit_price - entry_price) / entry_price * 100
                
                results.append({
                    'signal_date': signal_date,
                    'entry_price': entry_price,
                    'exit_date': exit_date,
                    'exit_price': exit_price,
                    'return_pct': return_pct,
                    'volume_ratio': signals.loc[signal_date, 'volume_ratio']
                })
        except:
            continue
    
    return pd.DataFrame(results)

# Backtest VCB Stopping Volume signals
sv_results = backtest_vpa_signals(vcb_vpa, 'stopping_volume', 5)
ns_results = backtest_vpa_signals(vcb_vpa, 'no_supply', 5)

print("📈 VCB Stopping Volume Backtest Results (5-day holding):")
if len(sv_results) > 0:
    print(f"Total signals: {len(sv_results)}")
    print(f"Average return: {sv_results['return_pct'].mean():.2f}%")
    print(f"Success rate (>0%): {(sv_results['return_pct'] > 0).mean()*100:.1f}%")
    print(f"Best trade: {sv_results['return_pct'].max():.2f}%")
    print(f"Worst trade: {sv_results['return_pct'].min():.2f}%")
else:
    print("No Stopping Volume signals found in the period")

print("\n📉 VCB No Supply Backtest Results (5-day holding):")
if len(ns_results) > 0:
    print(f"Total signals: {len(ns_results)}")
    print(f"Average return: {ns_results['return_pct'].mean():.2f}%")
    print(f"Success rate (>0%): {(ns_results['return_pct'] > 0).mean()*100:.1f}%")
    print(f"Best trade: {ns_results['return_pct'].max():.2f}%")
    print(f"Worst trade: {ns_results['return_pct'].min():.2f}%")
else:
    print("No Supply signals found in the period")

## 📝 Tóm Tắt và Những Bài Học Chính

### Những gì chúng ta đã hoàn thành:
1. ✅ Tải và phân tích dữ liệu thị trường chứng khoán Việt Nam thực tế
2. ✅ Tính toán các chỉ báo VPA thiết yếu từ dữ liệu OHLCV thô
3. ✅ Phát hiện tín hiệu VPA tự động bằng mã lệnh
4. ✅ Tham chiếu chéo với phân tích VPA chuyên gia
5. ✅ Thực hiện so sánh đa cổ phiếu theo ngành
6. ✅ Backtest hiệu suất tín hiệu

### Những Hiểu Biết Chính:
- **Mối quan hệ Khối lượng-Giá** là quan trọng cho phân tích thị trường
- **Phát hiện tự động** có thể xác định hầu hết các tín hiệu VPA chính
- **Phân tích chuyên gia** thêm bối cảnh quan trọng mà chỉ số không thể cung cấp
- **Thị trường Việt Nam** tuân theo các nguyên tắc VPA phổ quát
- **Backtesting** giúp xác thực hiệu quả tín hiệu

### Các Bước Tiếp Theo:
1. Tiếp tục với [Chương 1.2 - Luật Wyckoff](../chapter-1-2-wyckoff-laws.md)
2. Thực hành với các mẫu hình VPA nâng cao hơn
3. Học về các giai đoạn Tích lũy và Phân phối
4. Phát triển các hệ thống giao dịch toàn diện

---
*🎯 **Bài tập hoàn thành!** Bây giờ bạn đã có nền tảng vững chắc trong phân tích VPA với dữ liệu thị trường thực tế.*

## 🏠 Bài Tập Về Nhà

### Bài Tập 1: Phân Tích Mở Rộng
1. Chọn 3 cổ phiếu từ các ngành khác nhau (công nghệ, sản xuất, bán lẻ)
2. Áp dụng phân tích VPA để tìm tín hiệu mạnh nhất trong 3 tháng qua
3. So sánh đặc điểm ngành về mặt hành vi VPA

### Bài Tập 2: Tinh Chỉnh Tín Hiệu
1. Điều chỉnh tiêu chí phát hiện VPA để cải thiện độ chính xác
2. Kiểm tra các ngưỡng tỷ lệ khối lượng khác nhau
3. Thêm các bộ lọc bổ sung (vốn hóa thị trường, thanh khoản, v.v.)

### Bài Tập 3: Dự Án Nghiên Cứu
1. Đọc phân tích chuyên gia trong file `vpa_data/` cho 5 cổ phiếu khác nhau
2. Xác định các mẫu hình mà phân tích tự động đã bỏ sót
3. Đề xuất cải tiến cho thuật toán phát hiện

---
*📚 **Tài Nguyên:** Tiếp tục học tập với các tài nguyên VPA chính thức và thực hành với các mẫu hình phức tạp hơn trong các chương sắp tới.*